In [ ]:
import numpy as np 
import pandas as pd
from glob import glob
import time
from scipy.optimize import brute

from data_provider import read_data_path

In [2]:
def simulation(ticks, signals, init_data):
    market = init_data['market']
    commission = init_data['exchange_commision']
    start_capital = init_data['start_capital']
    buy_correction = init_data['buy_correction']
    sell_correction = init_data['sell_correction']
     
    cur_capital = start_capital 
    num_shares = 0
    trade_profit = 0
    cum_profit = 0
    
    market_position = 0
    start= len(ticks) - len(signals) 
    ticks = ticks[start:]
    trade_statistics = np.zeros((len(ticks), 7))
    for i in range(len(signals)) :
        
        if market_position == 0 and signals[i] == 1:
            price = ticks[i] * buy_correction
            num_shares = cur_capital * (1-commission)/price
            trade_profit = -cur_capital
            cur_capital = 0
            market_position = 1
            
        elif market_position == 1 and signals[i] == -1:
            price = ticks[i] * sell_correction
            cur_capital = (num_shares * price)*(1-commission)
            trade_profit = trade_profit + cur_capital
            cum_profit += trade_profit
            num_shares = 0
            market_position = 0
        else:
            signals[i] = 0

        trade_statistics[i] = [ticks[i], signals[i], market_position, cur_capital, num_shares, trade_profit, cum_profit]     

    df = pd.DataFrame(trade_statistics)
    df.columns = ['price', 'signal', 'MP', 'capital', 'num_shares', 'profit', 'cum_profit']
    df = df.astype({'signal':np.int32, 'MP':np.int32})
    return df

In [3]:
from robots_garage import ma_crossover_01

data_path = read_data_path()
markets = glob(data_path + '/BTC/*-BTC')

timeframes = ['30m']
init_data = {'exchange_commision': 0.0015,
             'start_capital'     : 0.1,
             'buy_correction'    : 1.004,
             'sell_correction'   : 0.996,
             'robot_name'        : 'ma_crossover_01',
             'market'            : '',
             'kline'             : ''  
            }
results = []

def trade_statistics(trade_data, init_data):
    #trade statistics        
    profit = trade_data.iloc[-1]['cum_profit']       

    s = trade_data['signal']
    trades = s[s==-1].shape[0] 
    
    ts = [init_data['market'], init_data['kline'], profit, trades]
    return ts

def fo(params):
    lead = params[0]
    lag = params[1]
    threshold = params[2]/1000
    
    signals = ma_crossover_01(prices, lead, lag, threshold) 
    ts = simulation(prices, signals, init_data)
    ts = trade_statistics(ts, init_data)
    
    results.append(ts + ['#'] + params.tolist() )
    return 0

In [ ]:
#Grid searsh
start_time = time.time()
for kline in timeframes:
    init_data['kline'] =  kline
    print(kline)
    results = []
    for market_path in markets:
        market = market_path.split('\\')[-1]
        init_data['market'] =  market
        print('\t', market)

        files = glob(market_path +  '/' + kline + '/' + '*csv') 
        df = pd.read_csv(files[-1])
        prices = df['C'].as_matrix()
        
        brute(fo, ((2,12,2), (60,100,2), (17,23,1)), finish=None)     
        xx = pd.DataFrame(results)
        xx.to_csv('Logs/mass_backtest_' + kline + '.csv', index = False)
end_time = time.time()
time_taken = (time.time() - start_time)/60
print("Time taken = {0:.3f}".format(time_taken),' min')

30m
	 EOS-BTC


In [ ]:
results = pd.DataFrame(results)
results.columns = ['market','kline','profit','ntrades','delim','lead','lag', 'threshold']
xx = results [results['profit'] > 0]
print('Всего результатов: ', xx.shape[0])
xx.groupby('market').sum()

In [37]:
timeframes = ['24m']
markets = ['EOS-BTC']

data_path = read_data_path()
start_time = time.time()
for kline in timeframes:
    init_data['kline'] =  kline
    print(kline)
    results = []
    equity = 0
    for market in markets:
        market_path  = data_path + '/BTC/' +market
        init_data['market'] =  market
        print('\t', market)

        files = glob(market_path +  '/' + kline + '/' + '*csv') 
        df = pd.read_csv(files[-1])
        prices = df['C'].as_matrix()
        
        signals = ma_crossover_01(prices, 8, 72, 0.018) 
        ts = simulation(prices, signals, init_data)
        ts.to_csv('Logs/trades_' + market + '_' + init_data['kline'] + '.csv', index = False)
        equity += ts['cum_profit']
        
end_time = time.time()
time_taken = (time.time() - start_time)/60
print("Time taken = {0:.3f}".format(time_taken),' min')

24m
	 EOS-BTC
Time taken = 0.004  min


array([[ 1.,  2.,  3.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])